# Loss Function

- Sum of Squares for Error, SSE
- Cross Entropy Error, CEE

## SSE : Sum of Squares for Error

In [1]:
import numpy as np

def sum_squares_error(y, t) :
    return 0.5 * np.sum((y - t) ** 2)

정답 2, 신경망 출력도 2

In [4]:
# 신경망의 출력(Softmax Function) -> 확률로 해석 가능
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] 
# 정답 레이블(one-hot encoding) : 2에 해당하는 원소 값이 1이므로 정답이 '2'임을 알 수 있다
t = [0, 0, 1, 0, 0, 0, 0, 0, 0 ,0] 

print(sum_squares_error(np.array(y), np.array(t)))

0.09750000000000003


정답 2, 신경망 출력은 7

In [5]:
# 신경망의 출력(Softmax Function) -> 확률로 해석 가능
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] 
# 정답 레이블(one-hot encoding) : 2에 해당하는 원소 값이 1이므로 정답이 '2'임을 알 수 있다
t = [0, 0, 1, 0, 0, 0, 0, 0, 0 ,0] 

print(sum_squares_error(np.array(y), np.array(t)))

0.5975


## CEE : Cross Entropy Error

In [8]:
# np.log()에 0을 입력하면 -inf가 되어 계산을 진행할 수 없기 때문에
# 아주 작은 값을 1e-7 == 0.0000001 더해서 0이 되지 않도록 해줘야 한다.
def cross_entropy_error(y, t) :
    delta = 1e-7 
    return -np.sum(t * np.log(y + delta))

정답 2, 신경망 출력도 2

In [9]:
# 신경망의 출력(Softmax Function) -> 확률로 해석 가능
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] 
# 정답 레이블(one-hot encoding) : 2에 해당하는 원소 값이 1이므로 정답이 '2'임을 알 수 있다
t = [0, 0, 1, 0, 0, 0, 0, 0, 0 ,0] 

print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338


정답 2, 신경망 출력은 7

In [10]:
# 신경망의 출력(Softmax Function) -> 확률로 해석 가능
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0] 
# 정답 레이블(one-hot encoding) : 2에 해당하는 원소 값이 1이므로 정답이 '2'임을 알 수 있다
t = [0, 0, 1, 0, 0, 0, 0, 0, 0 ,0] 

print(cross_entropy_error(np.array(y), np.array(t)))

2.302584092994546


# mini-batch

## MNIST Dataset Load

In [13]:
import numpy as np 
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # 6000만 장의 data 존재(하나의 data는 원래 28 * 28이었는데, flatten=True 때문에)
print(t_train.shape) 

(60000, 784)
(60000, 10)


## 훈련데이터, 시험데이터 랜덤 10장

In [19]:
train_size = x_train.shape[0] # 60,000장의 사진(데이터)
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print(batch_mask) #  10개 data의 index번호 random하게 뽑기

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

[40886 28863 16582 12850 39739 20213 54499 59357 50002 59470]


## batch용 CEE(Cross Entropy Error)

### 정답 레이블이 one-hot encoding일 경우

In [20]:
# y : 신경망 출력, t : 정답 레이블
# 데이터 하나당 Cross Entropy를 구하는 경우는 데이터의 형상을 바꿔준다.
def cross_entropy_error(y, t) :
    if y.ndim == 1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

### 정답 레이블이 숫자 레이블로 주어졌을 경우

In [21]:
# y : 신경망 출력, t : 정답 레이블
# 데이터 하나당 Cross Entropy를 구하는 경우는 데이터의 형상을 바꿔준다.
def cross_entropy_error(y, t) :
    if y.ndim == 1 :
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

# 정답에 해당하는 신경망의 출력만으로 교차 엔트로피 오차를 계산할 수 있다는 것이 핵심.
# 따라서 y[np.arange(batch_size), t]로 한 것이다.
# batch_size가 5, t=[2, 7, 0, 9, 4]라면?
# np.arange(batch_size, t)
# == y[np.arange([0, 1, 2, 3, 4]), [2, 7, 0, 9, 4]]]
# == [y[0, 2], y[1, 7], y[2, 0], y[3, 9], y[4, 4]] 